## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Guerrero Negro,MX,27.9769,-114.0611,85.46,42,81,10.78,broken clouds
1,1,Chatra,IN,24.2167,84.8667,73.38,91,93,3.51,overcast clouds
2,2,Wanlaweyn,SO,2.6185,44.8938,76.75,80,24,11.43,few clouds
3,3,Mehamn,NO,71.0357,27.8492,53.51,72,100,32.17,overcast clouds
4,4,Khatanga,RU,71.9667,102.5000,19.26,93,99,12.68,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                238
City                   238
Country                238
Lat                    238
Lng                    238
Max Temp               238
Humidity               238
Cloudiness             238
Wind Speed             238
Current Description    238
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Guerrero Negro,MX,85.46,broken clouds,27.9769,-114.0611,
2,Wanlaweyn,SO,76.75,few clouds,2.6185,44.8938,
7,Victoria,HK,84.33,overcast clouds,22.2855,114.1577,
10,Cabo San Lucas,MX,88.18,few clouds,22.8909,-109.9124,
11,Hithadhoo,MV,82.96,broken clouds,-0.6000,73.0833,
15,Road Town,VG,84.29,scattered clouds,18.4167,-64.6167,
17,Pierre,US,83.28,clear sky,44.3683,-100.3510,
25,Emerald,AU,77.34,clear sky,-23.5333,148.1667,
28,Price,US,76.71,clear sky,39.5994,-110.8107,
33,Port Blair,IN,80.11,few clouds,11.6667,92.7500,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Guerrero Negro,MX,85.46,broken clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
2,Wanlaweyn,SO,76.75,few clouds,2.6185,44.8938,GAASHAAN HOME
7,Victoria,HK,84.33,overcast clouds,22.2855,114.1577,Mini Hotel Central
10,Cabo San Lucas,MX,88.18,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
11,Hithadhoo,MV,82.96,broken clouds,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
692,Cozumel,MX,86.11,scattered clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort
694,Bairiki,KI,86.00,few clouds,1.3292,172.9752,Tarawa Boutique Hotel
695,Tutoia,BR,80.20,scattered clouds,-2.7619,-42.2744,Pousada Guará's
700,Kampot,KH,77.05,light rain,10.6167,104.1833,Two Moons Hotel


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</<dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
city_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))